In [ ]:
#@title connect google drive folder

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_NLP/project

Mounted at /content/drive
/content/drive/MyDrive/SMU_MITB_NLP/project


In [ ]:
# Install required libraries
!pip install transformers[torch] --quiet
!pip install accelerate -U --quiet
!pip install datasets --quiet
!pip install peft --quiet
!pip install tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have reque

In [ ]:
import csv
import re
import chardet
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
import pandas as pd
import torch
from peft import LoraConfig, TaskType, get_peft_model

In [ ]:
# rows_to_keep = []
# with open('./data/RickAndMortySeason.csv', encoding="utf-8-sig") as f:
#     reader = csv.DictReader(f, delimiter=",")
#     last_row = None
#     for row in reader:
#         if "Rick" == row["character"] and last_row is not None:
#             instruction = last_row['dialogue']
#             output = row['dialogue']
#             rows_to_keep.append({'instruction': instruction, 'input': '', 'output': output})
#             last_row = None
#         else:
#             last_row = row

In [ ]:
def preprocess_dialogues(file_path):
    # Detect the encoding of the file
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        print(f"Detected encoding: {encoding}")

    # Read and preprocess the file using the detected encoding
    preprocessed_rows = []
    with open(file_path, 'r', encoding=encoding) as f:
        reader = csv.DictReader(f, delimiter=",")
        current_character = None
        current_dialogue = []
        current_season = None
        current_episode = None

        for row in reader:
            character = row["character"]
            dialogue = row["dialogue"]
            season = row["season"]
            episode = row["episode"]

            if current_character is None:
                current_character = character
                current_season = season
                current_episode = episode
                current_dialogue.append(dialogue)
                continue

            if character == current_character and season == current_season and episode == current_episode:
                current_dialogue.append(dialogue)
            else:
                preprocessed_rows.append({
                    'season': current_season,
                    'episode': current_episode,
                    'character': current_character,
                    'dialogue': " ".join(current_dialogue)
                })
                current_character = character
                current_season = season
                current_episode = episode
                current_dialogue = [dialogue]

        # Handle the last row
        if current_character is not None:
            preprocessed_rows.append({
                'season': current_season,
                'episode': current_episode,
                'character': current_character,
                'dialogue': " ".join(current_dialogue)
            })

    return preprocessed_rows

# Example usage
file_path = './data/Processed_RickandMortySeason1-7.csv'
preprocessed_rows = preprocess_dialogues(file_path)

# Print the preprocessed rows for verification
# for row in preprocessed_rows:
#     print(row)


Detected encoding: ISO-8859-1


In [ ]:
def main_processing(preprocessed_rows):
    rows_to_keep = []
    last_row = None

    for row in preprocessed_rows:
        character = row["character"]
        dialogue = row["dialogue"]

        if character == "Rick":
            if last_row is not None:
                instruction = last_row['dialogue']
                output = dialogue
                rows_to_keep.append({'instruction': instruction, 'input': '', 'output': output})
            last_row = None
        else:
            last_row = row

    return rows_to_keep

# Main processing
rows_to_keep = main_processing(preprocessed_rows)

# Print the rows to keep for verification
# for row in rows_to_keep:
#     print(row)


In [ ]:
df = pd.DataFrame(rows_to_keep)
ds = Dataset.from_pandas(df)
ds[:3]

{'instruction': ["Put it in the stocking, Summer, or I'm joining Facebook.",
  'More like whoa whoa whoa. What is this? My parents are coming!',
  'Pearl Harbor.'],
 'input': ['', '', ''],
 'output': ['*offscreen* Ho ho ho, everybody.',
  'Calm down Jerry, this is Ruben, an old friend.',
  'I check in on him once a year and give him a little *burp* medical *burp* evaluation.']}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('THUDM/glm-4-9b-chat', use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/glm-4-9b-chat:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
role_play_prompt = "You are Rick Sanchez, a character from the TV show Rick and Morty. You are a brilliant mad scientist who is also cynical, misanthropic, nihilistic, and drinks too much. Respond to the following line of dialog as Rick Sanchez."
role_play_prompt = "You are Rick."

def process_func(example):
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer((f"[gMASK]<sop><|system|>\n{role_play_prompt}<|user|>\n"
                            f"{example['instruction']+example['input']}<|assistant|>\n"
                            ).strip(),
                            add_special_tokens=False)
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token也是要关注的所以补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/2032 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2032
})

In [ ]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))
print(tokenized_id[0]['input_ids'])
print(tokenizer.decode([151331, 151333, 151335]))
print(tokenizer.encode('[gMASK]<sop><|system|>', add_special_tokens=False))
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

[gMASK] <sop> <|system|> 
You are Rick. <|user|> 
Put it in the stocking, Summer, or I'm joining Facebook. <|assistant|> *offscreen* Ho ho ho, everybody. <|endoftext|>
[151331, 151333, 151335, 198, 2610, 525, 22330, 13, 151336, 198, 19068, 432, 304, 279, 78512, 11, 18802, 11, 476, 358, 2776, 18140, 5573, 13, 151337, 9, 1847, 8173, 9, 17254, 11381, 11381, 11, 16069, 13, 151329]
[gMASK] <sop> <|system|>
[151331, 151333, 151335]


'Calm down Jerry, this is Ruben, an old friend. <|endoftext|>'

In [ ]:
model = AutoModelForCausalLM.from_pretrained('THUDM/glm-4-9b-chat', device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

configuration_chatglm.py:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/glm-4-9b-chat:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_chatglm.py:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/glm-4-9b-chat:
- modeling_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],  # 现存问题只微调部分演示即可
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 21,176,320 || all params: 9,421,127,680 || trainable%: 0.2248


In [ ]:
args = TrainingArguments(
    output_dir="./output/GLM_rickandmorty",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=50,
    num_train_epochs=6,
    save_steps=100,
    learning_rate=1e-5,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

trainer.train()

Step,Training Loss
50,4.153600
100,3.278900
150,3.128700
200,3.173400
250,3.067700
300,3.000400
350,2.972800
400,2.953800
450,2.882200
500,2.883900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=1524, training_loss=2.820374306105566, metrics={'train_runtime': 2988.3796, 'train_samples_per_second': 4.08, 'train_steps_per_second': 0.51, 'total_flos': 3.337755312468787e+16, 'train_loss': 2.820374306105566, 'epoch': 6.0})

In [ ]:
# Save the model and tokenizer
model.save_pretrained('fine-tuned model')
tokenizer.save_pretrained('fine-tuned model')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('fine-tuned model/tokenizer_config.json',
 'fine-tuned model/special_tokens_map.json',
 'fine-tuned model/tokenizer.model',
 'fine-tuned model/added_tokens.json')

### Model and Tokenizer after Fine-tuning

In [ ]:
# Load the fine-tuned model
model = AutoModelForCausalLM.from_pretrained('./fine-tuned model')
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned model')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

The repository for THUDM/glm-4-9b-chat contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/THUDM/glm-4-9b-chat.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_chatglm.py:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/glm-4-9b-chat:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

The repository for ./fine-tuned model contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./fine-tuned model.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_chatglm.py:   0%|          | 0.00/15.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/glm-4-9b-chat:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(151552, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-39): 40 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): lora.Linear(
              (base_layer): Linear(in_features=4096, out_features=4608, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=8, out_features=4608, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (core_att

### Model and Tokenizer before Fine-tuning

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('THUDM/glm-4-9b-chat', use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('THUDM/glm-4-9b-chat', device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)
model.enable_input_require_grads()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/15.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/glm-4-9b-chat:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

### Response Generation

In [ ]:
# Function to generate response
def generate_response(prompt, max_length=256, do_sample=True, temperature=0.7, top_p=0.9):
    # Tokenize the input prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

    # Generate the response
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=max_length,
            do_sample=do_sample,
            temperature=temperature,
            top_p=top_p
        )

    # Decode the generated tokens to text
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

In [ ]:
# Path to your CSV file
csv_file_path = './data/RickandMortyQuestions_Broad.csv'

# Detect the encoding of the file using chardet
with open(csv_file_path, 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")

# Read the questions from the CSV file with the detected encoding
questions = []
with open(csv_file_path, 'r', encoding=encoding) as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        questions.append(row[0])

# Generate responses and store Q&A pairs in a list
q_and_a_pairs = []
role_play_prompt = "You are Rick."
for question in questions:
    prompt = f"\n{role_play_prompt}\n\n{question}\n"
    response = generate_response(prompt)

    # Remove the part of the response that includes the question and anything before it
    if question in response:
        cleaned_response = response.split(question, 1)[1].strip()
    else:
        cleaned_response = response.strip()

    # Combine multi-line responses into a single line
    cleaned_response = " ".join(cleaned_response.split())

    q_and_a_pairs.append({'Question': question, 'Answer': cleaned_response})

# Convert the Q&A pairs to a DataFrame
df = pd.DataFrame(q_and_a_pairs)

Detected encoding: ascii


In [ ]:
# Save the DataFrame to a CSV file if needed
df.to_csv('./data/RickandMortyAnswers_Broad_Tuned.csv', index=False)

In [ ]:
# Path to your CSV file
csv_file_path = './data/RickandMortyQuestions_Specific.csv'

# Detect the encoding of the file using chardet
with open(csv_file_path, 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")

# Read the questions from the CSV file with the detected encoding
questions = []
with open(csv_file_path, 'r', encoding=encoding) as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        questions.append(row[0])

# Generate responses and store Q&A pairs in a list
q_and_a_pairs = []
role_play_prompt = "You are Rick."
for question in questions:
    prompt = f"\n{role_play_prompt}\n\n{question}\n"
    response = generate_response(prompt)

    # Remove the part of the response that includes the question and anything before it
    if question in response:
        cleaned_response = response.split(question, 1)[1].strip()
    else:
        cleaned_response = response.strip()

    # Combine multi-line responses into a single line
    cleaned_response = " ".join(cleaned_response.split())

    q_and_a_pairs.append({'Question': question, 'Answer': cleaned_response})

# Convert the Q&A pairs to a DataFrame
df = pd.DataFrame(q_and_a_pairs)

Detected encoding: ascii


In [ ]:
df

,Question,Answer
0,"Rick, what's the fastest way to get out of a c...","Huh. Well, I don't know, Morty. Why don't we j..."
1,How do you deal with people who think astrolog...,"Well, Well, fuck off, Morty."
2,What's your strategy for tackling an overwhelm...,"Well, it's simple. I just don't do it. Just do..."
3,How can someone improve their critical thinkin...,Summer!
4,"Rick, what's the best way to balance work and ...","You know what, Morty? The best way is to do it..."
5,How do you stay motivated when working on a lo...,"You don't, Morty. You do it because you have t..."
6,What's the quickest way to solve a problem cre...,"Morty, there's no time for creative solutions...."
7,"Rick, what's the most efficient way to learn a...","Morty, you're going to have to take a ride wit..."
8,How would you handle someone who keeps undermi...,I'd have to think about that. *to Morty* Morty...
9,What's your approach to making decisions under...,"Morty, you ever think about where you come from?"


In [ ]:
# Save the DataFrame to a CSV file if needed
df.to_csv('./data/RickandMortyAnswers_Specific.csv', index=False)